# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [23]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

#Replace None by your own value and justify
OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(temperature=0.7, api_key=OPENAI_API_KEY)

In [13]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
    "Describe the following product: {product_type}"
)

In [24]:

chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [25]:
product = df['Product'][0] #Select a product type to be describe
chain.run(product)

/tmp/ipython-input-3137843778.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'A queen size sheet set typically includes a fitted sheet, a flat sheet, and two pillowcases, all designed to fit a queen size bed. The sheets are made of soft, comfortable materials such as cotton, microfiber, or linen, and come in a variety of colors and patterns. This set is designed to provide a luxurious and cozy sleeping experience for those with a queen size bed.'

## SimpleSequentialChain

In [26]:
from langchain.chains import SimpleSequentialChain

In [28]:
from google.colab import userdata
from langchain_openai import ChatOpenAI

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a review for the following product: {product_type}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [29]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following product review: {review}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [30]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [31]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
I recently purchased the Queen Size Sheet Set and I am extremely satisfied with my purchase. The sheets are incredibly soft and comfortable to sleep on, giving me a luxurious feel every night. The color I chose was a beautiful shade of blue that adds a pop of color to my bedroom.

The set comes with a fitted sheet, flat sheet, and two pillowcases, all of which fit perfectly on my queen size bed. The sheets have held up well after multiple washes and still look as good as new. I also appreciate that these sheets do not wrinkle easily, so my bed always looks neat and tidy.

Overall, I highly recommend the Queen Size Sheet Set to anyone in the market for new sheets. The quality is fantastic, the comfort level is top-notch, and the price is reasonable for the value you are receiving. I will definitely be purchasing more sets in different colors for my other bedrooms.
The reviewer purchased a Queen Size Sheet Set and is extremely satisfied wit

'The reviewer purchased a Queen Size Sheet Set and is extremely satisfied with their purchase. The sheets are soft, comfortable, and luxurious to sleep on, with a beautiful shade of blue that adds color to their bedroom. The set includes a fitted sheet, flat sheet, and two pillowcases that fit perfectly on their bed. The sheets have held up well after multiple washes and do not wrinkle easily. The reviewer highly recommends the set for its quality, comfort, and reasonable price, and plans to purchase more sets in different colors for their other bedrooms.'

**Repeat the above twice for different products**

## SequentialChain

In [32]:
from langchain.chains import SequentialChain

In [34]:
from google.colab import userdata
from langchain_openai import ChatOpenAI

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(temperature=0.9, api_key=OPENAI_API_KEY)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review to English: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="english_review" #Give a name to your output
                    )

In [38]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following product review: {english_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary" #give a name to this output
                    )

In [40]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review written in?: {review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [41]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "The original review was in {language}. The English translation is: {english_review}. Write a follow-up message based on this information."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [42]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "language", "followup_message"],
    verbose=True
)

In [43]:
review = df.Review[5]
overall_chain(review)

/tmp/ipython-input-1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the product, finding the taste to be mediocre and the foam not holding well. They suggest that the product may be an old batch or counterfeit, as they have had better experiences purchasing the same product in stores.',
 'language': 'French',
 'followup_message': 'We are sorry to hear about your disappointing experience with our product. We take the quality of our products very seriously and would like to investigate this issue further. Please provide us with more details such as the specific product you purchased, the batch number if available, and where you purchased it from. We want to ensu

In [63]:
review = df.Review[4]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'english_review': "I loved this product. But it seems to only last a few months. The company was great at replacing the first one (the frother falls out of the handle and can't be fixed). After 4 months, my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand, they will only replace it once. Anyway, if you have one, good luck.",
 'summary': 'The reviewer loved the product but was disappointed that it only lasted a few months before the frother fell ou

In [64]:
review = df.Review[3]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
 'english_review': '"This is the best throw pillow filler on Amazon. I have tried several others, and they are all cheap and flat no matter how much you fluff them. Once you put these in the dryer after removing them from the vacuum-sealed shipping material, they fluff up wonderfully."',
 'summary': 'The reviewer highly recommends this throw pillow filler, noting that it is superior to other options on Amazon. They found that other fillers were cheap and flat, but these ones fluff up nicely after being put in the dryer.',
 'language': 'English',
 'followup_message': "Thank you for sharing your positive experience with our throw pillow fillers! We're thrilled to hear that you find them to be the best on Amazon and tha

**Repeat the above twice for different products or reviews**

## Router Chain

In [44]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [45]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [46]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [48]:
from google.colab import userdata
from langchain_openai import ChatOpenAI

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(temperature=0, api_key=OPENAI_API_KEY)

In [49]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [50]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [51]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [52]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [53]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [54]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [55]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [56]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information, allowing for the transmission of traits from one generation to the next through the process of inheritance. This is essential for the survival and evolution of species, as it allows for genetic variation and adaptation to changing environments over time. \n\nIn summary, eve

In [58]:
chain.run("Why does my heart beat faster than my eyes?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does the heart beat faster than the eyes?'}
> Finished chain.


"The heart beats faster than the eyes because they serve different functions in the body. The heart is responsible for pumping blood throughout the body, delivering oxygen and nutrients to cells and removing waste products. In order to meet the body's demands for oxygen and nutrients during physical activity or times of stress, the heart rate increases to pump more blood.\n\nOn the other hand, the eyes are sensory organs that detect light and allow us to see. The speed at which the eyes process visual information is not as critical for survival as the heart's ability to deliver oxygen to tissues. Therefore, the heart beats faster than the eyes in order to meet the body's metabolic needs and maintain homeostasis."

In [59]:
chain.run("What does love do to the brain?")



> Entering new MultiPromptChain chain...
biology: {'input': 'What does love do to the brain?'}
> Finished chain.


'Love has a profound impact on the brain, triggering a complex cascade of chemical reactions and neural pathways. When we experience feelings of love, the brain releases a variety of neurotransmitters, including dopamine, oxytocin, and serotonin, which are associated with pleasure, bonding, and happiness.\n\nDopamine, often referred to as the "feel-good" neurotransmitter, is released in response to rewarding stimuli, such as spending time with a loved one or engaging in activities that bring us joy. This surge of dopamine can create feelings of euphoria and excitement, similar to the effects of drugs like cocaine.\n\nOxytocin, sometimes called the "love hormone," plays a key role in social bonding and attachment. It is released during intimate moments, such as hugging, kissing, or cuddling, and helps strengthen emotional connections between individuals. Oxytocin also promotes trust and empathy, fostering feelings of closeness and security in relationships.\n\nSerotonin, another neurotr

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [57]:
!pip install langchain_openai